In [ ]:
import import_ipynb
import os
import sys
import get_urls
from get_urls import *

In [ ]:
print(master_reports)
pd.DataFrame(master_reports).to_csv("master_reports.csv")

In [ ]:
#ANALYZE NAME CRITERIA FOR BALANCE SHEET, INCOME STATEMENT AND CASH FLOWS
#write unique short names to a csv
pd.DataFrame(unique(master_reports_extract['name_short'])).to_csv("stmt_name_list.csv")

In [ ]:
stmt_name_list = pd.read_csv("stmt_name_list.csv")
#print(unique(stmt_list))

In [ ]:
#BALANCE SHEET NAMES 
# consolidated balance sheet
# consolidated balance sheet (parenthetical)
# consolidated balance sheets
# consolidated balance sheets (parenthetical)
# consolidated balance sheets consolidated balance sheet parenthetical
# consolidated balance sheets parenthetical

bs_reports = master_reports_extract[master_reports_extract['name_short'].str.contains("consolidated balance sheet")]

In [ ]:
#INCOME STATEMENT NAMES 
# consolidated statements of comprehensive income
# consolidated statements of comprehensive income (loss)
# consolidated statements of comprehensive income (parenthetical)
# consolidated statements of comprehensive income (parentheticals)
# consolidated statements of comprehensive loss
# consolidated statements of earnings
# consolidated statements of earnings (parenthetical)
# consolidated statements of income
# consolidated statements of operations
# consolidated statements of operations and comprehensive income
# consolidated statements of operations and comprehensive income (loss)
# consolidated statements of operations and comprehensive income (parenthetical)
# consolidated statements of operations and comprehensive loss

ic_reports = master_reports_extract[master_reports_extract['name_short'].str.contains("consolidated statements of comprehensive income")
                                   | master_reports_extract['name_short'].str.contains("consolidated statements of operations and comprehensive income") 
                                   | master_reports_extract['name_short'].str.contains("consolidated statements of earnings") 
                                   ]


In [ ]:
# CASH FLOWS
# consolidated statements of cash flows
# consolidated statement of cash flows
# consolidated statements of cash flows (parenthetical)


cf_reports = master_reports_extract[master_reports_extract['name_short'].str.contains("consolidated statements of cash flows")
                                   | master_reports_extract['name_short'].str.contains("consolidated statement of cash flows") 
                                   ]
print(pd.DataFrame(cf_reports))

In [ ]:
#remove numbers and turn to float else none
def clean_data_str(string):
    result = re.sub('[^0-9]','', string)
    try:
        result = float(result)
    except Exception as ex:
        result = None 
        pass
    return(result)

In [ ]:
#give a list of statement urls and returns a list of dictionaries which have different parts of each statement (header, section, data)
def parse_statement_data(statements_url_df):
    # let's assume we want all the statements in a single data set.
    statements_data = []

    # loop through each statement url
    for index, row in statements_url_df.iterrows():

        # define a dictionary that will store the different parts of the statement.
        statement_data = {}
        statement_data['cik'] = row['cik']
        statement_data['name'] = row['name']
        statement_data['date'] = row['date']
        statement_data['headers'] = []
        statement_data['sections'] = []
        statement_data['data'] = []
        
        try:
            # request the statement file content
            #print("requesting statement: " + row['url'])
            content = requests.get(row['url']).content
            sleep(0.3)
            report_soup = BeautifulSoup(content, 'html')



            # find all the rows, figure out what type of row it is, parse the elements, and store in the statement file list.
            for index, row in enumerate(report_soup.table.find_all('tr')):

                # first let's get all the elements.
                cols = row.find_all('td')

                # if it's a regular row and not a section or a table header
                if (len(row.find_all('th')) == 0 and len(row.find_all('strong')) == 0): 
                    reg_row = [ele.text.strip() for ele in cols]
                    statement_data['data'].append(reg_row)

                # if it's a regular row and a section but not a table header
                elif (len(row.find_all('th')) == 0 and len(row.find_all('strong')) != 0):
                    sec_row = cols[0].text.strip()
                    statement_data['sections'].append(sec_row)

                # finally if it's not any of those it must be a header
                elif (len(row.find_all('th')) != 0):            
                    hed_row = [ele.text.strip() for ele in row.find_all('th')]
                    statement_data['headers'].append(hed_row)

                else:            
                    print('We encountered an error.')
        except Exception as ex:
            print("Failed")
            pass

        # append it to the master list.
        statements_data.append(statement_data)
    statement_df = pd.DataFrame(statements_data)
   # print(statement_df)
    for index, row in statement_df.iterrows():
        for i in row['data']:
            try: 
                metric_cik = row['cik']
                metric_company = row['name']
                metric_year = row['headers'][1][0][-4:]
                metric_name = i[0]
                metric_value = clean_data_str(i[1])
                #print(metric_cik)
                #print(metric_name)
                # print(metric_value)
            except Exception as ex:
                pass
    